In [1]:
import os
import numpy as np

## Ejercicio stream cipher

In [ ]:
def generate_keystream(length, seed=None):
    if seed is not None:
        np.random.seed(seed)
    return np.random.randint(0, 256, length, dtype=np.uint8)

In [3]:
def xor_encrypt_decrypt(message, keystream):
    
    return bytes([m ^ k for m, k in zip(message, keystream)])

In [5]:
mensaje_original = "Esto es una prueba".encode()  # Convertimos el mensaje a bytes
seed = 42  # Semilla para reproducibilidad
keystream = generate_keystream(len(mensaje_original), seed)

# Cifrado
mensaje_cifrado = xor_encrypt_decrypt(mensaje_original, keystream)
print(f"Mensaje cifrado: {mensaje_cifrado}")

# Descifrado
mensaje_descifrado = xor_encrypt_decrypt(mensaje_cifrado, keystream)
print(f"Mensaje descifrado: {mensaje_descifrado.decode()}")  # Convertimos de bytes a string


Mensaje cifrado: b'#\xaf\x950\x93X\x99\xeb)m\x03\xd3~\xe7\x80K\x08\x95'
Mensaje descifrado: Esto es una prueba


## Pruebas unitarias

In [ ]:
import unittest

class TestXORCipher(unittest.TestCase):
    
    def test_keystream_length(self):
        #Verifica que el keystream generado tenga la longitud correcta
        message_length = 20
        keystream = generate_keystream(message_length, seed=42)
        self.assertEqual(len(keystream), message_length)

    def test_xor_encrypt_decrypt(self):
        #Verifica que cifrar y luego descifrar un mensaje devuelve el original
        message = "Prueba de cifrado".encode()
        seed = 123  # Usamos una semilla para pruebas reproducibles
        keystream = generate_keystream(len(message), seed)

        encrypted_message = xor_encrypt_decrypt(message, keystream)
        decrypted_message = xor_encrypt_decrypt(encrypted_message, keystream)

        self.assertEqual(decrypted_message, message)  # Debe coincidir con el original

    def test_xor_symmetry(self):
        #Verifica que aplicar XOR dos veces con el mismo keystream devuelve el original
        data = b"Mensaje secreto"
        key = generate_keystream(len(data), seed=99)

        encrypted = xor_encrypt_decrypt(data, key)
        decrypted = xor_encrypt_decrypt(encrypted, key)

        self.assertEqual(decrypted, data)

    def test_different_keys_produce_different_results(self):
        #Verifica que usar keystreams diferentes da resultados diferentes
        message = "Hola mundo!".encode()
        key1 = generate_keystream(len(message), seed=1)
        key2 = generate_keystream(len(message), seed=2)

        encrypted1 = xor_encrypt_decrypt(message, key1)
        encrypted2 = xor_encrypt_decrypt(message, key2)

        self.assertNotEqual(encrypted1, encrypted2)  # Los cifrados deben ser diferentes



In [7]:
unittest.main(argv=[''], verbosity=2, exit=False)

test_different_keys_produce_different_results (__main__.TestXORCipher.test_different_keys_produce_different_results)
Verifica que usar keystreams diferentes da resultados diferentes. ... ok
test_keystream_length (__main__.TestXORCipher.test_keystream_length)
Verifica que el keystream generado tenga la longitud correcta. ... ok
test_xor_encrypt_decrypt (__main__.TestXORCipher.test_xor_encrypt_decrypt)
Verifica que cifrar y luego descifrar un mensaje devuelve el original. ... ok
test_xor_symmetry (__main__.TestXORCipher.test_xor_symmetry)
Verifica que aplicar XOR dos veces con el mismo keystream devuelve el original. ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.015s

OK



# Análisis de Seguridad del Keystream en un Cifrado XOR

## 1. ¿Qué sucede cuando cambias la clave utilizada para generar el keystream?  
Cuando se cambia la clave (seed/nonce) utilizada para inicializar el generador de números pseudoaleatorios (PRNG), se obtiene un keystream completamente diferente. Esto significa que el mensaje cifrado con una clave determinada no podrá ser descifrado con otra clave distinta, ya que el keystream usado en el cifrado y descifrado debe ser exactamente el mismo.  

## 2. ¿Qué riesgos de seguridad existen si reutilizas el mismo keystream para cifrar dos mensajes diferentes?  
Si reutilizas el mismo keystream para cifrar dos mensajes distintos, un atacante puede compararlos y descubrir información sobre los mensajes originales.

Esto sucede porque en el cifrado XOR, cada letra del mensaje se combina con el keystream de la misma manera. Si dos mensajes distintos usan el mismo keystream, un atacante que tenga ambos mensajes cifrados puede hacer un cálculo simple para eliminar el keystream y obtener una relación directa entre los mensajes originales.

## 3. ¿Cómo afecta la longitud del keystream a la seguridad del cifrado?  
  
- **Debe ser al menos del mismo tamaño que el mensaje:** Si el keystream es más corto, se repetirá, lo que permite a un atacante identificar patrones y debilitar el cifrado, similar a reutilizar la misma clave.  

- **Lo ideal es que sea tan largo como el mensaje:** Si cada bit del mensaje se cifra con un bit único del keystream, se obtiene una mejor seguridad, similar a un cifrado de un solo uso (One-Time Pad), que es teóricamente irrompible si el keystream es completamente aleatorio y usado solo una vez.  

- **Si es demasiado corto o predecible:** Un atacante podría analizar los mensajes cifrados y descubrir el keystream, lo que le permitiría descifrar futuros mensajes o incluso reconstruir los originales. Esto es especialmente peligroso si el keystream se genera con un método poco seguro o una clave predecible.

### ¿Qué debes tener en cuenta al generar un keystream en un entorno real?  

- **Usar un generador de números aleatorios seguro:** Es importante que el keystream sea realmente aleatorio. Para ello, se deben usar fuentes seguras como `os.urandom()` (para efectos de este ejercicio en python), en lugar de generadores débiles como `rand()` o `numpy.random`, que pueden ser predecibles.  

- **No reutilizar claves:** Si el keystream se genera a partir de una clave, cada mensaje debe usar un valor único para evitar que un atacante descubra patrones y comprometa la seguridad del cifrado.  

- **Asegurar la sincronización en sistemas distribuidos:** Si el cifrado y descifrado ocurren en diferentes dispositivos, es fundamental que ambos usen exactamente el mismo keystream. De lo contrario, el mensaje no podrá descifrarse correctamente.  

